In [2]:
import requests
import json
import os

import pandas as pd
# import seaborn as sns

from glob import glob
from tqdm import tqdm
import multiprocessing


import numpy as np
from torch_geometric.utils.convert import from_networkx

# from androguard.core.bytecodes.apk import APK
from androguard.core.apk import APK
import networkx as nx
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed
from androguard.misc import AnalyzeAPK
import matplotlib.pyplot as plt

import torch
from torch_geometric.datasets import MalNetTiny
import torch_geometric.transforms as T
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Read file 

In [5]:
pos_path = '/projects/academic/erdem/jakir/easybuild/Ahmet/top_subreddits_unbanned_incel_users.txt'
neg_path = '/projects/academic/erdem/jakir/easybuild/Ahmet/top_subreddits_unbanned_non_incel_users.txt'


unique_sub_reddit, map_sub_red, user_cnt = set(), {}, 0

all_data = [[], []]
all_data_y = []


def read_lines(filepath, pos=True):
    global user_cnt, all_data, all_data_y, map_sub_red, unique_sub_reddit
    cnt= 0
    with open(filepath, "r") as f:
        for line in f:
#             if cnt > 2:
#                 break
            
            cnt += 1
#             print(cnt)
            cur_line = line.strip()
            cur_list = cur_line.split(': ')
            
            if len(cur_list) == 1:
                continue
                
            if pos:
                all_data[1].append(1)
            else:
                all_data[1].append(0)
            feat_array = np.zeros(9433)
                
#             print(user_id)
            feat_list = cur_list[1].split(',')
            
            for feat in feat_list: 
#                 print(feat)
                split_text = feat.split(" (", 1)
                sub_red = split_text[0]
                count = split_text[1].rstrip(")")
                unique_sub_reddit.add(sub_red)
                
                if sub_red not in map_sub_red:
                    map_sub_red[sub_red] = user_cnt
                    user_cnt += 1
                
                feat_array[map_sub_red[sub_red]] = count
                
            all_data[0].append(feat_array)


read_lines(pos_path, pos=True)
read_lines(neg_path, pos=False)

print(len(unique_sub_reddit), user_cnt)



9433 9433


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  # Example classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report, roc_curve, roc_auc_score, accuracy_score, precision_recall_fscore_support

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier



print(len(all_data[0]))
print(all_data[1][0])

X_train, X_test, y_train, y_test = train_test_split(all_data[0], all_data[1], test_size=0.2, random_state=42)


2324
1


In [31]:
# print(len(X_train),  len(X_test))

def ML_model(model='Regression'):
    if model=='Regression':
        model = LogisticRegression()
    elif model == 'SVM':
        model = SVC(kernel='rbf')
    elif model == 'DecisionTree':
        model = DecisionTreeClassifier()
    elif model == 'RandomForest':
        model = RandomForestClassifier(n_estimators=100)
    elif model == 'KNN':
        model = KNeighborsClassifier(n_neighbors=5)
    
    model.fit(X_train, y_train)
    return model

# model =  ML_model(model='Regression')
# y_pred = model.predict(X_test)

In [32]:
all_models = ['Regression', 'SVM', 'DecisionTree', 'RandomForest', 'KNN']

for model in all_models:
    model =  ML_model(model=model)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    print(model,  "  Accuracy:", accuracy, f1)


    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

#     print("Classification Report:\n", report)
#     print("Confusion Matrix:\n", cm)

/cvmfs/soft.ccr.buffalo.edu/versions/2023.01/easybuild/software/avx512/MPI/gcc/11.2.0/openmpi/4.1.1/scikit-learn/1.0.2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()   Accuracy: 0.7548387096774194 0.7548285048285048
SVC()   Accuracy: 0.7010752688172043 0.7008760603662516
DecisionTreeClassifier()   Accuracy: 0.7182795698924731 0.7173960445188796
RandomForestClassifier()   Accuracy: 0.7935483870967742 0.7934710193204529
KNeighborsClassifier()   Accuracy: 0.6559139784946236 0.648167063897401
